# Finding Similar Items: Textually Similar Documents

In [64]:
# pip install docx2txt

In [65]:
import os, sys
import docx2txt
import random
import itertools
import hashlib

In [66]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [67]:
file_source = '/content/drive/My Drive/..'

In [68]:
docPathOne = os.path.abspath(file_source+"...docx") 
docOne = docx2txt.process(docPathOne)
docPathTwo = os.path.abspath(file_source+"...docx")
docTwo = docx2txt.process(docPathTwo)
docPathThree = os.path.abspath(file_source+"...docx")
docThree = docx2txt.process(docPathThree)

In [69]:
class Shingle:
    def __init__(self, k, doc):
        self.k = k
        self.doc = self.splitDoc(doc)
        self.hashes = self.shingle()
        
    def splitDoc(self, doc):
        # splits the words of the document and stores each to a field of a list
        return doc.split()

    def shingle(self):
        # goes through the list of words of the document and adds k-shingles to a set
        # eof = end of file
        eof = False
        container = set()
        i = 0
        # goes as long eof is not reached
        while not eof:
            end = min(self.k, len(self.doc)-i) #not used
            # tuples are hashable
            shingledValues = tuple(self.doc[i:i+self.k])
            container.add(hash(shingledValues))
            # jumps ahead k fields in list
            i += self.k
            # cheks if eof is reached
            try: self.doc[i]
            except: eof = True

        return container

In [70]:
def jaccardSimilarity(s1,s2):
    return len(s1.hashes & s2.hashes) / len(s1.hashes | s2.hashes)

In [71]:
class MinHashing:
    def __init__(self, sets):
        # sets should be a tuple of sets
        self.sets = sets
        # store number of sets for iterators
        self.numberSets = len(self.sets)
        # store a set of global unique hashes
        self.globalHashes = self._addSets()
        # method calles to create signatures of for each set
        self._defineSignatures()
        # empty container for minHashMatrix, that will be filled with a method that has to be called
        self.minHashMatrix = []

    def _addSets(self):
        resultSet = set()
        # adding values of one set to another with |= operator
        for sets in self.sets:
            resultSet |= sets.hashes
        return resultSet

    def _defineSignatures(self):
        # create an empty dictionary for each document / set to store the signature in it
        self.signatures = {i: {} for i in range(self.numberSets)}
        # go through each of the global hash list ...
        for element in self.globalHashes:
            # ... and compare with each set for occurence in this set of hashes
            for idx, sets in enumerate(self.sets):
                self.signatures[idx][element] = element in sets.hashes

    def _findMinHashes(self, permutedKeys):
        # create a list of zeros that should later receive the minHashValues for each of the documents
        values = [0] * self.numberSets
        i = 0
        # iterate as long as not all values are assigned a value
        while not all(values):
            # iterate through the number of sets to see if the element occurs in the matrix, then assign number of iteration
            for idx in range(self.numberSets):
                if self.signatures[idx][permutedKeys[i]] and not values[idx]: values[idx] = i      
            i += 1
            if i > 1000:
                break
        return values
    
    def setMinHashMatrix(self, n):
        # whenever the method is called, the minHashMatrix will be reset to an empty list
        self.minHashMatrix = []
        # n : number of permutations
        # store global hash list to a new variable, as this will be shuffled
        hashList = list(self.globalHashes)
        # repeat for all permutations that are set by the user
        # append result of _findMinHashes method that will be given the shuffled hash list to matrix
        for i in range(n):
            random.shuffle(hashList)
            self.minHashMatrix.append(self._findMinHashes(hashList))
            
    def setSimilarities(self, lsh=False, nBands=1, threshold=.8):
        if not lsh:
            # create a number of integers for each document
            # then permute the number so we receive all combinations of two document id's to compare
            listSets = [i for i in range(self.numberSets)]
            setPermutations = itertools.permutations(listSets, 2)
            
            # create a dictionary that will contain a similarity value for each of the possible doc id combinations
            # to create dictionary we used dict comprehension dict = {... for ... in ...}
            self.docSimilarities = {
                # key is the combination e.g. doc id (0) compared to (->) doc id (1)
                '{}'.format(combination[0])+'->{}'.format(combination[1]): \
                self._calcSimilarities(combination, 0, len(self.minHashMatrix)) for combination in setPermutations
            }
        
        if lsh:
            listSets = [i for i in range(self.numberSets)]
            setPermutations = itertools.permutations(listSets, 2)

            # assign starting rows of buckets according to number of buckets and length of minHashMatrix
            bands = list(range(0, len(self.minHashMatrix)+1, int(len(self.minHashMatrix)/nBands)))
            # pick a random bucket, avoid last band as it may be too small
            b = int(random.random()*len(bands)-1)

            docSimilaritiesBucket = {
                '{}'.format(combination[0])+'->{}'.format(combination[1]): \
                self._calcSimilarities(combination, bands[b], bands[b+1]) for combination in setPermutations
            }
            self.docSimilarities = {key: 1 if value < threshold else 0 for key, value in docSimilaritiesBucket.items()}
    
    def _calcSimilarities(self, combination, firstRow, lastRow):
        a = 0
        if lastRow == len(self.minHashMatrix):
            lastRow -= 1
        # go through each row of the minHashMatrix
        for i in range(firstRow, lastRow+1):
            # compare the columns according to the permuation given with the doc ids
            # a counts up, if a similar entry existed
            if self.minHashMatrix[i][combination[0]] == self.minHashMatrix[i][combination[1]]:
                a += 1
        # return the ratio between equal entries and all entries as the similarity score        
        return a / (lastRow-firstRow+1)        

In [72]:
one = Shingle(k=2, doc=docOne)
two = Shingle(k=2, doc=docTwo)
three = Shingle(k=2, doc=docThree)
four = Shingle(k=2, doc=docOne)

minhasher = MinHashing((one, two, three, four))
minhasher.setMinHashMatrix(100)
minhasher.setSimilarities(lsh=False, nBands=10)
minhasher.docSimilarities

{'0->1': 0.06,
 '0->2': 0.06,
 '0->3': 1.0,
 '1->0': 0.06,
 '1->2': 1.0,
 '1->3': 0.06,
 '2->0': 0.06,
 '2->1': 1.0,
 '2->3': 0.06,
 '3->0': 1.0,
 '3->1': 0.06,
 '3->2': 0.06}

The following is an alterantive implementation where shingles are constructed by letter

In [73]:
class Shingling:
  def construct_Shingles(document, k):
    # loop through the document and break it based on k-chars
    shingles = set()
    idx = 0
    for sh in range(len(document) - k + 1):
      #adds = []
      #shingles.add()
      shingles.add(document[sh:(sh+k)])
      idx += k
    return shingles

document = 'abcdef'
k = 2
print(Shingling.construct_Shingles(document,k))

{'cd', 'ef', 'bc', 'ab', 'de'}


In [74]:
#pip install mmh3

In [75]:
import mmh3

class CompareSets:
  def construct_Shingles_Hashed(document, k, seed=0):
    # loop through the document and break it based on k-chars
    shingles = set()
    for sh in range(len(document) - k + 1):
      shingles.add(str(mmh3.hash(document[sh:(sh+k)], seed)))
    return shingles

  def calculate_JaccardSimularity(set1, set2):
      # Jaccord similarity is defined as 
      # [(elements in intersection) / (elements in union)]  of the two sets 
      result = len(set1 & set2) / len(set1 | set2) # Python's intersection function is & and union is |
      return result

In [76]:
document_1 = 'ABCDE'
document_2 = 'ABCDEF'
k = 2

shingle_1_without_hash = Shingling.construct_Shingles(document_1, k)
shingle_2_without_hash = Shingling.construct_Shingles(document_2, k)

shingle_1 = CompareSets.construct_Shingles_Hashed(document_1, k)
shingle_2 = CompareSets.construct_Shingles_Hashed(document_2, k)

print("Shingles:")
print("Document 1: " + str(shingle_1_without_hash))
print("Document 2: " + str(shingle_2_without_hash))
print('Jaccard Similarity:', CompareSets.calculate_JaccardSimularity(shingle_1_without_hash,shingle_2_without_hash))

print()
print("Shingles (hashed):")
print("Document 1: " + str(shingle_1))
print("Document 2: " + str(shingle_2))
print('Jaccard Similarity:', CompareSets.calculate_JaccardSimularity(shingle_1,shingle_2))

print()

print("When Jaccard Similarity equals 1 then the documents are identical")

Shingles:
Document 1: {'CD', 'AB', 'DE', 'BC'}
Document 2: {'CD', 'EF', 'AB', 'DE', 'BC'}
Jaccard Similarity: 0.8

Shingles (hashed):
Document 1: {'-200631661', '-1515746955', '286687249', '1928019024'}
Document 2: {'-1515746955', '566201179', '1928019024', '-200631661', '286687249'}
Jaccard Similarity: 0.8

When Jaccard Similarity equals 1 then the documents are identical
